In [3]:
class Interval:
    def __init__(self,start,end):
        self.start = start
        self.end = end
    def __repr__(self):
        return f"[{self.start},{self.end}]"

### Merge all the overlapping intervals to produce a list that has only mutually exclusive intervals.
* Four possible cases : do not overlap, a completely overlaps b, same start time, diff ends, b starts after a starts and ends after a
* ‘a’ overlaps ‘b’ means b.start <= a.end == > merge into c:
    c.start = a.start
    c.end = max(a.end, b.end)
* O(NlogN) for sort + O(N) for iteration ~ O(NlogN)

In [6]:
def merge_intervals(intervals):
    if len(intervals) < 2:
        return intervals
    
    intervals.sort(key=lambda x:x.start) #sort by start time
    
    merged = []
    start = intervals[0].start
    end = intervals[0].end
    
    for i in range(1,len(intervals)):
        print(intervals[i])
        if intervals[i].start <= end: #if overlapping, continue iteration until end is found
            end = max(end,intervals[i].end)
        else:
            start = intervals[i].start # not overlapping, append to result as is
            end = intervals[i].end
            merged.append(intervals[i])
    merged.append(Interval(start, end))
    return merged

In [7]:
merge_intervals([Interval(1,4),Interval(2,6),Interval(3,5)])

[2,6]
[3,5]


[[1,6]]

### Given a list of non-overlapping intervals sorted by their start time, insert a given interval at the correct position and merge all necessary intervals to produce a list that has only mutually exclusive intervals.
* Find position to insert by iterating over intervals and skipping all where end < new.start
* Find overlapping intervals with the new one, merge into exclusive interval
* Insert remaining intervals
* O(N)

In [41]:
def insert_and_merge_intervals(intervals, new):
    merged = []
    i, start, end = 0, 0, 1
    while i < len(intervals) and intervals[i][end]<new[start]:
        merged.append(intervals[i])
        i+=1
    while i < len(intervals) and intervals[i][start]<=new[end]:
        new[start] = min(intervals[i][start], new[start])
        new[end] = max(intervals[i][end], new[end])
        i+=1

    merged.append(new)
    while i < len(intervals):
        merged.append(intervals[i])
        i+=1
    return merged
        
        

In [42]:
insert_and_merge_intervals([[1,3], [5,7], [8,12]], [4,6])

[[1, 3], [4, 7], [8, 12]]

### Given two lists of intervals, find the intersection of these two lists. 
Each list consists of disjoint intervals sorted on their start time.

1. a overlaps b if start of b lies between start and end of a.
2. b overlaps a if b starts between interval a.
3. Intersection is given by max(a.start, b.start) & min(a.end,b.end)
* O(N+M)



In [43]:
def find_intersection(interval1, interval2):
    results = []
    i, j, start, end = 0, 0, 0, 1
    while i < len(interval1) and j < len(interval2):
        
        a_overlaps_b = interval1[i][start] <= interval2[j][start] and interval1[i][start] >= interval2[j][end]
        b_overlaps_a = interval1[i][start] >= interval2[j][start] and interval1[i][start] <= interval2[j][end]
        
        if a_overlaps_b or b_overlaps_a:
            results.append(
                [max(interval1[i][start],interval2[j][start]), min(interval1[i][end], interval2[j][end])])
        if interval1[i][end] < interval2[j][end]:
            i+=1
        else:
            j+=1
    return results

In [45]:
find_intersection([[1, 3], [5, 6], [7, 9]], [[2, 3], [5, 7]])

[[5, 6], [7, 7]]

### Given an array of intervals representing ‘N’ appointments, find out if a person can attend all the appointments.
1. Sort by start time
2. Appointments will conflict if one begins before the previous one starts
* O(NlogN)

In [49]:
def find_conflicts(intervals):
    i, start, end = 1,0,1
    intervals.sort(key=lambda x:x[0])
    while i < len(intervals):
        if intervals[i][start] < intervals[i-1][end]:
            return False
        i+=1
    return True
        

In [51]:
print(find_conflicts([[1,4], [2,5], [7,9]]))
print(find_conflicts([[6,7], [2,4], [8,12]]))
print(find_conflicts([[4,5], [2,3], [3,6]]))

False
True
False


### Given a list of intervals representing the start and end time of ‘N’ meetings, find the minimum number of rooms required to hold all the meetings.
-- Or time of trains and required platforms
1. Use a minheap to keep track of ongoing meetings
2. Will need as many rooms as the max length of minheap at any point
3. Pop meetings if end time <= start of current meeting
* O(NlogN)

In [61]:
from heapq import *
class Meeting:
    def __init__(self, start, end):
        self.start = start
        self.end = end
    def __lt__(self, other):
        return self.end < other.end

def find_rooms(meetings):
    meetings.sort(key=lambda x:x.start)
    min_rooms = 0
    minheap = []
    heapify(minheap)
    for meeting in meetings:
        while len(minheap) > 0 and meeting.start >= minheap[0].end:
            heappop(minheap)
        heappush(minheap, meeting)
        minrooms = max(min_rooms, len(minheap))
    return minrooms        

In [62]:
find_rooms([Meeting(2,3), Meeting(4,5), Meeting(2,4), Meeting(3,5)])

2

### Given a list of Jobs with Start time, an End time, and a CPU load, find the maximum CPU load at any time if all the jobs are running on the same machine.
- Sort by start time
- Use a min heap to keep track of all current jobs
- Pop if jobs have ended
- Push current job
- Add current load to those in min heap at each step
- Keep track of max at each step
* O(NLogN)

In [65]:
from heapq import *
class Job:
    def __init__(self, start, end, load):
        self.start = start
        self.end = end
        self.load = load
    def __lt__(self, other):
        return self.end < other.end

def max_job_load(jobs):
    jobs.sort(key=lambda x:x.start)
    minheap = []
    heapify(minheap)
    curr_load, max_load = 0, 0
    for job in jobs:
        while len(minheap) > 0 and job.start >= minheap[0].end:
            curr_load-= minheap[0].load
            heappop(minheap)
        heappush(minheap, job)
        curr_load+=job.load
        max_load = max(curr_load, max_load)
    return max_load        

In [66]:
max_job_load([Job(1,4,3), Job(2,5,4), Job(7,9,6)])

7

#### For ‘K’ employees, we are given a list of intervals representing the working hours of each employee. Find out if there is a free interval that is common to all employees. 
- You can assume that each list of employee working hours is sorted on the start time.
- Combine into one list and sort by start time
- If not overlapping i.e start of b > end of a, add interval(a.end, b.start) as free to the results
* O(NlogN) + O(N*N) + O(N) = O(N*N)
- Better approach would be to use minheap and utilize the sorted sublist of intervals.

In [77]:
def find_free_intervals(intervals):
    intervals = [i for emp in intervals for i in emp]
    
    intervals.sort(key=lambda x:x.start) #sort by start time
    
    results = []
    start = intervals[0].start
    end = intervals[0].end
    
    for i in range(1,len(intervals)):
        if intervals[i].start <= end: #if overlapping, continue iteration until end is found
            end = max(end,intervals[i].end)
        else:
            results.append(Interval(end,intervals[i].start))

    return results

In [78]:
find_free_intervals([[Interval(1,3), Interval(2,4)], [Interval(3,5), Interval(7,9)]])

[[5,7]]